In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [ ]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [ ]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [ ]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

NextValidId: 1


In [ ]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [ ]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

# query the current portfolio


In [ ]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

In [ ]:
last_portfolio

In [ ]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


Total value of the account:  1000210.66
end of account summary


In [ ]:
total_value

1000210.66

# implementation
* decide what to buy & sell, as well as the quantity

In [ ]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 14 Price: 5419.0
Tick Price. Ticker Id: 0 tickType: 6 Price: 5500.0
Tick Price. Ticker Id: 0 tickType: 7 Price: 5330.51
Tick Price. Ticker Id: 0 tickType: 9 Price: 5357.92
Tick Price. Ticker Id: 0 tickType: 4 Price: 5459.78
Tick Price. Ticker Id: 1 tickType: 4 Price: 2469.19
Tick Price. Ticker Id: 1 tickType: 6 Price: 2484.79
Tick Price. Ticker Id: 1 tickType: 7 Price: 2443.7
Tick Price. Ticker Id: 1 tickType: 9 Price: 2445.74
Tick Price. Ticker Id: 1 tickType: 14 Price: 2464.08
Tick Price. Ticker Id: 1 tickType: 1 Price: 2469.47
Tick Price. Ticker Id: 1 tickType: 2 Price: 2472.28
Tick Price. Ticker Id: 2 tickType: 14 Price: 1669.56
Tick Price. Ticker Id: 2 tickType: 6 Price: 1693.97
Tick Price. Ticker Id: 2 tickType: 7 Price: 1666.03
Tick Price. Ticker Id: 2 tickType: 9 Price: 1652.56
Tick Price. Ticker Id: 2 tickType: 4 Price: 1693.0
Tick Price. Ticker Id: 2 tickType: 1 Price: 1692.63
Tick Price. Ticker Id: 2 tickType: 2 Price: 1694.71
Tick Price. T

Tick Price. Ticker Id: 9 tickType: 1 Price: 381.92
Tick Price. Ticker Id: 9 tickType: 2 Price: 383.13


In [ ]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [ ]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.9) / target_portfolio['mkt_price'])

In [ ]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL


In [ ]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,NVR,0.1,5357.92,18.0
1,BKNG,0.1,2445.74,40.0
2,CMG,0.1,1652.56,60.0
3,AZO,0.1,2495.68,40.0
4,TSLA,0.1,181.41,550.0
5,URI,0.1,454.90,219.0
6,SIVB,0.1,313.38,318.0
7,REGN,0.1,758.00,131.0
8,NOW,0.1,473.81,210.0
9,MKTX,0.1,377.00,264.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [ ]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = last_portfolio_stock['quantity'].values[0]
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['ticker'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['ticker'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [ ]:
stocks_to_buy

[['NVR', 18.0],
 ['URI', 219.0],
 ['CMG', 60.0],
 ['TSLA', 550.0],
 ['REGN', 131.0],
 ['NOW', 210.0],
 ['AZO', 40.0],
 ['BKNG', 40.0],
 ['MKTX', 264.0],
 ['SIVB', 318.0]]

In [ ]:
stocks_to_sell

[]

# place order

In [ ]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 1
openOrder. orderId:2, contract:755864,NVR,STK,,0,?,,SMART,,USD,NVR,NVR,False,,,,combo:, order:2,1000,1927015624: MKT BUY 18@0 GTC
orderStatus. orderId: 2, status: Submitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:1927015624, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:2, contract:755864,NVR,STK,,0,?,,SMART,,USD,NVR,NVR,False,,,,combo:, order:2,1000,1927015624: MKT BUY 18@0 GTC
orderStatus. orderId: 2, status: Submitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:1927015624, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:3, contract:3629755,URI,STK,,0,?,,SMART,,USD,URI,URI,False,,,,combo:, order:3,1000,1927015625: MKT BUY 219@0 GTC
orderStatus. orderId: 3, status: Submitted, filled: 0, remaining:219, avgFillPrice: 0.0, permId:1927015625, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:4, contract:37655664,CMG,ST

openOrder. orderId:5, contract:76792991,TSLA,STK,,0,?,,SMART,,USD,TSLA,NMS,False,,,,combo:, order:5,1000,1927015627: MKT BUY 550@0 GTC
orderStatus. orderId: 5, status: Submitted, filled: 403, remaining:147, avgFillPrice: 193.115037, permId:1927015627, parentId:0, lastFillPrice: 193.11, clientId: 1000, whyHeld: , mktCapPrice:0.0
CommissionReport. ExecId: 00025b46.63dbd136.01.01, Commission: 0.5, Currency: USD, RealizedPnL: , Yield: , YieldRedemptionDate: 0
openOrder. orderId:5, contract:76792991,TSLA,STK,,0,?,,SMART,,USD,TSLA,NMS,False,,,,combo:, order:5,1000,1927015627: MKT BUY 550@0 GTC
orderStatus. orderId: 5, status: Submitted, filled: 403, remaining:147, avgFillPrice: 193.115037, permId:1927015627, parentId:0, lastFillPrice: 193.11, clientId: 1000, whyHeld: , mktCapPrice:0.0
CommissionReport. ExecId: 00025b46.63dbd137.01.01, Commission: 0.5, Currency: USD, RealizedPnL: , Yield: , YieldRedemptionDate: 0
openOrder. orderId:8, contract:4750,AZO,STK,,0,?,,SMART,,USD,AZO,AZO,False,,,,co